In [0]:

// Databricks notebook: Get all jobs finished in the last 30 minutes (Scala)
import scala.util.parsing.json._
import scala.io.Source
import java.time.Instant
import java.time.ZoneId
import java.time.format.DateTimeFormatter
import java.net.HttpURLConnection
import java.net.URL

// CONFIG: update these
val DATABRICKS_INSTANCE = "https://adb-1042629664363467.7.azuredatabricks.net"
val TOKEN = dbutils.secrets.get(scope="taxi-scope", key="key")

val HEADERS = Map(
  "Authorization" -> s"Bearer $TOKEN",
  "Content-Type" -> "application/json"
)

// TIME RANGE: last 30 minutes
val nowMillis = System.currentTimeMillis()
val thirtyMinutesAgoMillis = nowMillis - 72*60*60*1000

// API CALL: list job runs
val url = new URL(s"$DATABRICKS_INSTANCE/api/2.1/jobs/runs/list")
val conn = url.openConnection().asInstanceOf[HttpURLConnection]
conn.setRequestMethod("GET")
HEADERS.foreach { case (k,v) => conn.setRequestProperty(k,v) }

val response = {
  val inputStream = if (conn.getResponseCode < 400) conn.getInputStream else conn.getErrorStream
  Source.fromInputStream(inputStream).mkString
}

// PARSE JSON
val data = JSON.parseFull(response).get.asInstanceOf[Map[String, Any]]
val runs = data.getOrElse("runs", List.empty).asInstanceOf[List[Map[String, Any]]]

// FILTER: runs finished in last 30 minutes
val recentRuns = runs.filter { run =>
  val endTime = run.getOrElse("end_time", 0.0).asInstanceOf[Double].toLong
  endTime >= thirtyMinutesAgoMillis && endTime <= nowMillis
}

// OUTPUT
val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm:ss").withZone(ZoneId.systemDefault())

if (recentRuns.nonEmpty) {
  println(s"Found ${recentRuns.size} job runs finished in the last 30 minutes:\n")
  recentRuns.foreach { run =>
    val jobId = run.getOrElse("job_id", "N/A")
    val runId = run.getOrElse("run_id", "N/A")
    val endTime = formatter.format(Instant.ofEpochMilli(run.getOrElse("end_time", 0.0).asInstanceOf[Double].toLong))
    val state = run.get("state").flatMap(_.asInstanceOf[Map[String, Any]].get("result_state")).getOrElse("N/A")
    println(s"Job ID: $jobId, Run ID: $runId, Finished: $endTime, State: $state")
  }
} else {
  println("No jobs finished in the last 30 minutes.")
}

